<div style="background-color: black;">
<hr style="border: 3px solid skyblue;">
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue; text-transform: uppercase;">
    TIME SERIES DATA PROCESSING
    <br>
    LOAD REAL TIME | LOAD DAY AHEAD
</div>
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
Fulfilling Missed Time Steps Notebook
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
        Each part of the following script was used for the pacially processed time series i.e. Availability Factors, Scalled Inflows, Current Load and Load Dayahead data, to fill al those missing time steps, or point points that do not content any value or falling that, content N/A, n/e or '0' value.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
    <hr style="border: 2px solid skyblue;">
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [13]:
import pandas as pd
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError  # Ensure this line is included
from entsoe.exceptions import NoMatchingDataError, InvalidBusinessParameterError  # Import relevant exceptions
import os
import csv
from datetime import datetime
import pytz
from pytz import timezone, utc
import shutil
from functools import lru_cache
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime, timedelta

<div style="background-color: black;">
<hr style="border: 2px solid skyblue;">
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
- If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="background-color: black;">
<hr style="border: 2px solid skyblue;">
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    3. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: Justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Indicate the year of all data is referring to in the variable data_year.
<br>
The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
<br>Additionally there are some default parameters that has to be defined to the correct working and calling to the ENTSO-E downloading functions.
</div>

In [3]:
# Year to which data refers to:
data_years_list = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [4]:
# Additional string to be appended
#availability_factors_folder_path = dispaSET_unleash_folder_path + "\Database\AvailabilityFactors"
load_dayahead_folder_path = dispaSET_unleash_folder_path + "/Database/Load_DayAhead"
load_realtime_folder_path = dispaSET_unleash_folder_path + "/Database/Load_RealTime"
#scalled_inflows_folder_path = dispaSET_unleash_folder_path + "\Database\HydroData\ScaledInflows"

#print("availability_factors_folder_path:", availability_factors_folder_path)
print("load_dayahead_folder_path:", load_dayahead_folder_path)
print("load_realtime_folder_path:", load_realtime_folder_path)
#print("scalled_inflows_folder_path:", scalled_inflows_folder_path)

load_dayahead_folder_path: /home/ray/Dispa-SET_Unleash/Database/Load_DayAhead
load_realtime_folder_path: /home/ray/Dispa-SET_Unleash/Database/Load_RealTime


<div style="background-color: black;">
<hr style="border: 2px solid skyblue;">
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    4. Missed Values Fulfilling
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Because of the data from the original sources, there are some time points that might have the value of '0'.
<br>
This missed points are completed using the data from previous years.
</div>

In [9]:
directory_path = load_realtime_folder_path

def is_leap_year(year):
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def adjust_year_in_timestamp(timestamp, new_year):
    dt = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S+00:00')
    if dt.month == 2 and dt.day == 29 and not is_leap_year(new_year):
        return dt.replace(year=new_year, day=28).strftime('%Y-%m-%d %H:%M:%S+00:00')
    return dt.replace(year=new_year).strftime('%Y-%m-%d %H:%M:%S+00:00')

def load_csv_files(directory_path, data_years_list):
    data_frames = {}
    for year in data_years_list:
        file_path = os.path.join(directory_path, f"{year}.csv")
        if os.path.exists(file_path):
            # Read CSV file with no header
            df = pd.read_csv(file_path, header=None)
            df.columns = df.columns.astype(str)  # Convert column names to strings
            data_frames[year] = df
        else:
            print(f"{file_path} does not exist, skipping...")
    return data_frames

def find_value_in_cached_data(target_timestamp, data_frames, target_year, data_years_list, search_previous=True):
    current_year_index = data_years_list.index(target_year)
    search_range = range(current_year_index - 1, -1, -1) if search_previous else range(current_year_index + 1, len(data_years_list))

    for index in search_range:
        other_year = data_years_list[index]
        if other_year in data_frames:
            df_other = data_frames[other_year]
            adjusted_timestamp = adjust_year_in_timestamp(target_timestamp, other_year)
            row = df_other[df_other[df_other.columns[0]] == adjusted_timestamp]
            if not row.empty:
                value = row.iloc[0][df_other.columns[1]]
                if value > 0:
                    return value
    return None

def process_csv_file(file_path, data_frames, data_years_list, year):
    if os.path.exists(file_path):
        print(f"Processing file: {file_path}")
        # Read CSV file with no header
        df = pd.read_csv(file_path, header=None)
        df.columns = df.columns.astype(str)  # Convert column names to strings

        zero_value_rows = df[df[df.columns[1]] == 0]
        updates = []

        for idx, row in zero_value_rows.iterrows():
            target_timestamp = row[df.columns[0]]
            new_value = find_value_in_cached_data(target_timestamp, data_frames, year, data_years_list, search_previous=True)
            if new_value is None:
                new_value = find_value_in_cached_data(target_timestamp, data_frames, year, data_years_list, search_previous=False)
            if new_value is not None:
                updates.append((idx, new_value))

        if updates:
            for idx, new_value in updates:
                df.at[idx, df.columns[1]] = new_value
            df.to_csv(file_path, index=False, header=False)  # Save without headers
            print(f"File {file_path} updated successfully.")
    else:
        print(f"File {file_path} does not exist, skipping...")

def process_csv_files_in_directory(directory_path, data_years_list):
    data_frames = load_csv_files(directory_path, data_years_list)
    for year in data_years_list:
        file_path = os.path.join(directory_path, f"{year}.csv")
        process_csv_file(file_path, data_frames, data_years_list, year)

def process_all_csv_files(load_realtime_folder_path, data_years_list):
    for root, dirs, files in os.walk(load_realtime_folder_path):
        for subdir in ['1h', '30min', '15min']:
            subdir_path = os.path.join(root, subdir)
            if os.path.exists(subdir_path):
                print(f"Processing CSV files in folder: {subdir_path}")
                process_csv_files_in_directory(subdir_path, data_years_list)
            else:
                print(f"Sub-folder {subdir} does not exist in {root}, skipping...")

process_all_csv_files(load_realtime_folder_path, data_years_list)

Sub-folder 1h does not exist in /home/ray/Dispa-SET_Unleash/Database/Load_RealTime, skipping...
Sub-folder 30min does not exist in /home/ray/Dispa-SET_Unleash/Database/Load_RealTime, skipping...
Sub-folder 15min does not exist in /home/ray/Dispa-SET_Unleash/Database/Load_RealTime, skipping...
Processing CSV files in folder: /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h
Processing file: /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h/2016.csv
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h/2016.csv updated successfully.
Processing file: /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h/2017.csv
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h/2017.csv updated successfully.
Processing file: /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h/2018.csv
File /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h/2018.csv updated successfully.
Processing file: /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h/2019.csv
File /

<div style="background-color: black;">
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
There remain some missed points that do not have any value even for all the analized years.
<br>
This missed points are completed using the data from the closest time step in the same hour inside the analized year.
</div>

In [22]:
# Function to replace 0 values in the second column
def replace_zeros(df):
    df_copy = df.copy()
    time_col = 0  # Index of the time column (first column)
    data_col = 1  # Index of the data column (second column)

    # Convert time column to datetime for easier manipulation
    df_copy[time_col] = pd.to_datetime(df_copy[time_col])

    # Sort values by datetime in case they are not sorted
    df_copy.sort_values(by=time_col, inplace=True)
    
    for index, row in df_copy.iterrows():
        if row[data_col] == 0:
            current_time = row[time_col]
            same_hour = df_copy[(df_copy[time_col].dt.hour == current_time.hour) & 
                                (df_copy[time_col].dt.minute == current_time.minute)]

            # Get future values first, skip the current row's date
            future_values = same_hour[same_hour[time_col] > current_time][data_col]
            non_zero_future = future_values[future_values > 0]

            if not non_zero_future.empty:
                df_copy.at[index, data_col] = non_zero_future.iloc[0]
            else:
                # No future non-zero values, check the past values
                past_values = same_hour[same_hour[time_col] < current_time][data_col]
                non_zero_past = past_values[past_values > 0]
                if not non_zero_past.empty:
                    df_copy.at[index, data_col] = non_zero_past.iloc[-1]  # Take the latest available past value
    
    return df_copy

# Main function to iterate over all directories and process CSV files
def process_files(data_years_list, file_path_1):
    for year in data_years_list:
        for root, dirs, files in os.walk(file_path_1):
            for dir_name in dirs:
                if dir_name in ['1h', '30min', '15min']:
                    dir_path = os.path.join(root, dir_name)
                    
                    # Check if the CSV for the current year exists in this subfolder
                    csv_file = f"{year}.csv"
                    csv_path = os.path.join(dir_path, csv_file)
                    
                    if os.path.exists(csv_path):
                        # Read the CSV file without headers
                        df = pd.read_csv(csv_path, header=None)
                        
                        # Process the DataFrame to replace zeros
                        df_updated = replace_zeros(df)
                        
                        # Save the updated DataFrame back to CSV (overwrite the original file)
                        df_updated.to_csv(csv_path, index=False, header=False)
                        print(f"Processed {csv_path}")


file_path_1 = load_realtime_folder_path # Replace with the actual directory path

process_files(data_years_list, file_path_1)

Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/IE/30min/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/CZ/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/NL/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/NL/30min/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/NL/15min/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/PT/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/EL/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/LT/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/NO/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/LV/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/SI/1h/2016.csv
Processed /home/ray/Dispa-SET_Unleash/Database/Load_RealTime/SK/1h/2016.csv
Pro